In [91]:
import pandas as pd
import os

testcase_dir = r"D:\crwu\data\foodname_classification\测试集V2.1\已标注\已评审"

testcase_paths = [
    os.path.join(testcase_dir, file)
    for file in os.listdir(testcase_dir)
    if file.endswith(".xlsx") and file.startswith("C")
]
testcase_paths.sort(reverse=True)


ref_target_df = pd.DataFrame(
    {
        "原始菜名": [],
        "建议菜名": [],
        "是否删除": [],
    }
)

for file in testcase_paths:
    print(f"读取文件：{file}")
    if not (
        (
            os.path.basename(file).startswith("C5")
            and os.path.basename(file).endswith("merged.xlsx")
        )
        or os.path.basename(file).startswith("C3")
    ):
        continue
    df = pd.read_excel(file, sheet_name=0)
    # print(df)
    temp_df = pd.DataFrame(
        {
            "原始菜名": [],
            "建议菜名": [],
            "是否删除": [],
        }
    )
    temp_df["原始菜名"] = df["菜品名"]
    temp_df["建议菜名"] = df["建议菜名"]
    temp_df["是否删除"] = df["是否删除"]
    add_list = df[df["替换菜名"].notnull()]["替换菜名"].tolist()
    print(add_list)
    add_df = pd.DataFrame(
        {
            "原始菜名": add_list,
            "建议菜名": [pd.NA for _ in add_list],
            "是否删除": [False for _ in add_list],
        }
    )
    temp_df = pd.concat([temp_df, add_df])
    print(temp_df.tail(10))
    temp_df.reset_index(drop=True, inplace=True)
    for i in range(len(temp_df)):
        if pd.isna(temp_df.iloc[i]["建议菜名"]):
            temp_df.loc[i, "建议菜名"] = temp_df.iloc[i]["原始菜名"]
        if pd.isna(temp_df.iloc[i]["原始菜名"]) and not pd.isna(temp_df.iloc[i]["建议菜名"]):
            temp_df.loc[i, "原始菜名"] = temp_df.iloc[i]["建议菜名"]
        if pd.isna(temp_df.iloc[i]["是否删除"]) or temp_df.iloc[i]["是否删除"] == 0:
            temp_df.loc[i, "是否删除"] = False
        else:
            temp_df.loc[i, "是否删除"] = True
    print(temp_df.tail(10))

    ref_target_df = pd.concat([ref_target_df, temp_df])
    # print(temp_df)
ref_target_df.to_excel(
    os.path.join(
        r"D:\crwu\data\foodname_classification\测试集V2.1\已标注\已评审", "ref.xlsx"
    ),
    index=False,
)
print("保存文件：ref.xlsx")

读取文件：D:\crwu\data\foodname_classification\测试集V2.1\已标注\已评审\C5_testcase_main_food_freq4_familarity5_yl.xlsx
读取文件：D:\crwu\data\foodname_classification\测试集V2.1\已标注\已评审\C5_testcase_main_food_freq4_familarity5_merged.xlsx
['雪菜肉丝面', '过桥米线', '扬州炒饭', '法棍面包', '墨鱼水饺', '担担面', '西班牙海鲜饭', '烤地瓜', '榨菜肉丝面', '土豆牛腩饭', '烤包子']
      原始菜名 建议菜名  是否删除
1     过桥米线  NaN   0.0
2     扬州炒饭  NaN   0.0
3     法棍面包  NaN   0.0
4     墨鱼水饺  NaN   0.0
5      担担面  NaN   0.0
6   西班牙海鲜饭  NaN   0.0
7      烤地瓜  NaN   0.0
8    榨菜肉丝面  NaN   0.0
9    土豆牛腩饭  NaN   0.0
10     烤包子  NaN   0.0
       原始菜名    建议菜名   是否删除
401    过桥米线    过桥米线  False
402    扬州炒饭    扬州炒饭  False
403    法棍面包    法棍面包  False
404    墨鱼水饺    墨鱼水饺  False
405     担担面     担担面  False
406  西班牙海鲜饭  西班牙海鲜饭  False
407     烤地瓜     烤地瓜  False
408   榨菜肉丝面   榨菜肉丝面  False
409   土豆牛腩饭   土豆牛腩饭  False
410     烤包子     烤包子  False
读取文件：D:\crwu\data\foodname_classification\测试集V2.1\已标注\已评审\C3_testcase_memory_freq3_familarity4_crw.xlsx
['东北大拉皮', '麻婆豆腐盖饭', '佛跳墙']
       原始菜名  建议菜名  是否删除

C:\Users\crwu\AppData\Local\Temp\ipykernel_11184\2057813357.py:64: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  temp_df.loc[i, "是否删除"] = True
c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\crwu\AppData\Local\Temp\ipykernel_11184\2057813357.py:62: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  temp_df.loc[i, "是否删除"] = False


In [ ]:
for file in testcase_paths:
    print(f"读取文件：{file}")
    if not os.path.basename(file).startswith("C5"):
        continue
    df = pd.read_excel(file, sheet_name=0)
    ref_target_df = pd.DataFrame(
        {
            "爬取菜名": [],
            "ground_truth": [],        
        }
    )
    